In [ ]:
import pandas as pd
import numpy as np


In [55]:
# Charger les fichiers
freq = pd.read_csv("../data/raw/fremtpl2/freMTPL2freq.csv")
sev = pd.read_csv("../data/raw/fremtpl2/freMTPL2sev.csv")

# 1️⃣ Lignes avec ClaimNb = 0
freq_no_claim = freq[freq["ClaimNb"] == 0].copy()
freq_no_claim["ClaimAmount"] = 0

# 2️⃣ Lignes avec ClaimNb > 0 ET présentes dans sev
freq_with_claim = freq[freq["ClaimNb"] > 0].copy()
freq_with_claim = freq_with_claim.merge(sev, on="IDpol", how="inner")  # inner join pour ne garder que ceux présents dans sev

# 3️⃣ Concaténer les deux
policy_master = pd.concat([freq_no_claim, freq_with_claim], ignore_index=True)

# 4️⃣ Vérification rapide
print(policy_master.shape)
print(policy_master["ClaimAmount"].head(10))
print(policy_master["ClaimNb"].value_counts())
policy_master.head()
# 5️⃣ Sauvegarde (optionnel)
#policy_master.to_csv("data/processed/policy_master.csv", index=False)


(670397, 13)
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: ClaimAmount, dtype: float64
ClaimNb
0     643953
1      23570
2       2597
3        186
11        22
4         20
16        16
5         10
9          9
8          8
6          6
Name: count, dtype: int64


,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
0,24952.0,0,0.005464,A,6,15,50,50,B2,Diesel,29,R24,0.0
1,24953.0,0,0.670000,A,6,15,50,50,B2,Diesel,29,R24,0.0
2,24955.0,0,0.890000,A,6,8,30,68,B1,Diesel,48,R53,0.0
3,24956.0,0,0.100000,A,6,8,30,57,B1,Diesel,48,R53,0.0
4,24958.0,0,0.200000,B,7,4,44,50,B14,Diesel,56,R24,0.0


In [ ]:
freq.groupby("claim_occurred").describe()

IDpol                                                  \
                   count          mean           std      min        25%   
claim_occurred                                                             
0               643953.0  2.672656e+06  1.624084e+06  24952.0  1178973.0   
1                34060.0  1.661431e+06  1.678434e+06      1.0    23273.5   

                                                  ClaimNb            ...  \
                      50%        75%        max     count      mean  ...   
claim_occurred                                                       ...   
0               2279137.0  4056988.0  6114330.0  643953.0  0.000000  ...   
1               1137080.0  3088999.5  6113971.0   34060.0  1.059953  ...   

               BonusMalus          Density                                 \
                      75%    max     count         mean          std  min   
claim_occurred                                                              
0                    64.0  230.0  643953.0  1783.206055  3942.416958  1.0   
1                    72.0  228.0   34060.0  1966.670669  4250.156926  2.0   

                                               
                  25%    50%     75%      max  
claim_occurred                                 
0                91.0  388.0  1629.0  27000.0  
1               101.0  448.0  1955.0  27000.0  

[2 rows x 64 columns]

In [57]:
policy_master["claim_occurred"] = (policy_master["ClaimNb"] > 0).astype(int)
policy_master.groupby("claim_occurred")["ClaimAmount"].describe()

,count,mean,std,min,25%,50%,75%,max
claim_occurred,,,,,,,,
0,643953.0,0.000000,0.000000,0.0,0.0000,0.0,0.000,0.00
1,26444.0,2265.512649,29371.031113,1.0,685.9925,1172.0,1212.385,4075400.56


In [42]:
policy_master.head()

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,claim_occurred
0,24952.0,0,0.005464,A,6,15,50,50,B2,Diesel,29,R24,0.0,0
1,24953.0,0,0.670000,A,6,15,50,50,B2,Diesel,29,R24,0.0,0
2,24955.0,0,0.890000,A,6,8,30,68,B1,Diesel,48,R53,0.0,0
3,24956.0,0,0.100000,A,6,8,30,57,B1,Diesel,48,R53,0.0,0
4,24958.0,0,0.200000,B,7,4,44,50,B14,Diesel,56,R24,0.0,0


In [58]:
import pandas as pd

# policy_master = ton dataframe complet

# séparer les classes
claims = policy_master[policy_master["claim_occurred"] == 1]
no_claims = policy_master[policy_master["claim_occurred"] == 0]

# undersampling des non-sinistres
no_claims_sampled = no_claims.sample(n=len(claims), random_state=42)

# recombiner
policy_balanced = pd.concat([claims, no_claims_sampled], ignore_index=True)

# mélanger les lignes pour éviter tout ordre biaisé
policy_balanced = policy_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# vérifier la proportion
print(policy_balanced["claim_occurred"].value_counts())


claim_occurred
1    26444
0    26444
Name: count, dtype: int64


In [59]:
policy_balanced.head()

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,claim_occurred
0,1115411.0,2,0.89,E,5,18,29,112,B2,Regular,4348,R72,564.12,1
1,1161295.0,1,0.90,F,5,6,36,90,B1,Regular,16291,R11,14871.57,1
2,5060934.0,0,0.62,C,6,4,25,85,B3,Diesel,197,R93,0.00,0
3,1134355.0,0,1.00,A,6,6,56,50,B6,Diesel,18,R54,0.00,0
4,4132878.0,1,1.00,A,8,14,27,85,B2,Diesel,42,R72,1204.00,1


In [64]:
import numpy as np

rng = np.random.default_rng(42)

# 1. Shop area
base_area = df["VehPower"] * 10
df["shop_area_m2"] = (
    base_area * rng.lognormal(mean=0, sigma=0.15, size=len(df))
).clip(8, 220)

# 2. Activity type (probabilistic but constrained)
def activity_from_area(area):
    if area < 45:
        return rng.choice(["kiosk", "cafe"], p=[0.6, 0.4])
    elif area < 80:
        return rng.choice(["cafe", "grocery"], p=[0.3, 0.7])
    elif area < 120:
        return rng.choice(["grocery", "clothing"], p=[0.4, 0.6])
    else:
        return rng.choice(["clothing", "pharmacy"], p=[0.3, 0.7])

df["activity_type"] = df["shop_area_m2"].apply(activity_from_area)

# 3. Assets
activity_mult = {
    "kiosk": 0.6,
    "cafe": 0.8,
    "grocery": 1.0,
    "clothing": 1.1,
    "pharmacy": 1.4,
}

df["assets_value_tnd"] = (
    df["shop_area_m2"] *
    700 *
    df["activity_type"].map(activity_mult) *
    rng.lognormal(0, 0.20, len(df))
)

# 4. Revenue
density_norm = np.log1p(df["Density"]) / np.log1p(df["Density"].max())
margin = rng.uniform(0.10, 0.14, len(df))
df["revenue_monthly_tnd"] = df["assets_value_tnd"] * margin * (0.9 + 0.2 * density_norm)

# 5. Revenue bucket (quantiles)
df["revenue_bucket"] = pd.qcut(
    df["revenue_monthly_tnd"],
    q=3,
    labels=["low", "medium", "high"]
)

In [65]:
df.head(10)

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,...,claim_occurred,shop_area_m2,activity_type,assets_value_tnd,revenue_monthly_tnd,revenue_bucket,open_at_night,security_alarm,security_camera,claim_cost_tnd
0,1115411.0,2,0.89,E,5,18,29,112,B2,Regular,...,1,52.338413,cafe,32039.867178,3435.241967,low,True,False,False,564.12
1,1161295.0,1,0.90,F,5,6,36,90,B1,Regular,...,1,42.778062,cafe,20571.217759,2934.531494,low,True,False,False,14871.57
2,5060934.0,0,0.62,C,6,4,25,85,B3,Diesel,...,0,67.148880,grocery,48596.971039,6280.478157,high,True,True,False,0.00
3,1134355.0,0,1.00,A,6,6,56,50,B6,Diesel,...,0,69.091331,grocery,41102.814180,5504.684487,medium,True,True,False,0.00
4,4132878.0,1,1.00,A,8,14,27,85,B2,Diesel,...,1,59.702348,cafe,33442.666824,3824.101673,low,False,True,True,1204.00
5,1150883.0,1,0.66,B,4,8,30,60,B5,Regular,...,1,32.902628,kiosk,14111.362634,1393.828153,low,True,False,False,48.13
6,31265.0,1,1.00,A,4,12,40,50,B6,Regular,...,1,40.774444,cafe,23842.093896,2907.992308,low,True,False,False,1128.12
7,5010143.0,0,0.01,E,5,2,44,50,B12,Regular,...,0,47.683557,cafe,26634.685261,3681.739198,low,True,False,False,0.00
8,5006679.0,0,0.29,D,5,5,55,50,B12,Regular,...,0,49.874150,grocery,55884.617828,7397.222599,high,True,False,False,0.00
9,3162273.0,1,0.80,A,9,0,43,50,B10,Diesel,...,1,79.190242,cafe,32121.613549,3952.987736,low,False,True,True,1204.00


In [67]:


rng = np.random.default_rng(42)

df = policy_master.copy()

# ----------------------------
# 0) Years active (maturity)
# ----------------------------
# Exposure is usually in years in freMTPL2 (often <= 1). We'll keep it, and clip.
df["years_active"] = df["Exposure"].astype(float).clip(0.05, 5.0)

# Create a maturity score in [0,1] (simple scaling)
# If Exposure mostly in [0,1], this is basically maturity within first year.
df["maturity"] = (df["years_active"] / df["years_active"].quantile(0.95)).clip(0, 1)

# ----------------------------
# 1) Shop area (continuous)
# ----------------------------
base_area = df["VehPower"].astype(float) * 10
# Add controlled noise + a mild maturity boost (mature shops tend to be larger or better organized)
df["shop_area_m2"] = (
    base_area
    * rng.lognormal(mean=0, sigma=0.15, size=len(df))
    * (0.95 + 0.20 * df["maturity"])
).clip(8, 220)

# ----------------------------
# 2) Activity type (fuzzy borders + maturity)
# ----------------------------
# Mature businesses slightly more likely to be "pharmacy/clothing" for same size.
def activity_from_area_and_maturity(area, m):
    # area drives the base category; maturity nudges upward slightly
    if area < 45:
        return rng.choice(["kiosk", "cafe"], p=[0.65 - 0.15*m, 0.35 + 0.15*m])
    elif area < 80:
        return rng.choice(["cafe", "grocery"], p=[0.35 - 0.10*m, 0.65 + 0.10*m])
    elif area < 120:
        return rng.choice(["grocery", "clothing"], p=[0.50 - 0.20*m, 0.50 + 0.20*m])
    else:
        return rng.choice(["clothing", "pharmacy"], p=[0.40 - 0.15*m, 0.60 + 0.15*m])

df["activity_type"] = [
    activity_from_area_and_maturity(a, m)
    for a, m in zip(df["shop_area_m2"].values, df["maturity"].values)
]

# ----------------------------
# 3) Assets value (area × activity × maturity + noise)
# ----------------------------
activity_assets_mult = {
    "kiosk": 0.6,
    "cafe": 0.8,
    "grocery": 1.0,
    "clothing": 1.1,
    "pharmacy": 1.4,
}

base_assets = df["shop_area_m2"] * 700 * df["activity_type"].map(activity_assets_mult).astype(float)

# maturity increases assets (inventory + equipment) moderately
df["assets_value_tnd"] = (
    base_assets
    * (0.85 + 0.40 * df["maturity"])
    * rng.lognormal(0, 0.18, len(df))
).clip(500, 600000)

# ----------------------------
# 4) Revenue monthly (assets × turnover_rate + density + maturity)
# ----------------------------
density_norm = np.log1p(df["Density"].astype(float)) / np.log1p(df["Density"].astype(float).max())

# turnover rate depends on activity (simple and explainable)
turnover_rate = {
    "kiosk": 0.11,
    "cafe": 0.10,
    "grocery": 0.12,
    "clothing": 0.09,
    "pharmacy": 0.13,
}
base_turn = df["activity_type"].map(turnover_rate).astype(float).values

# add small random variation around base turnover
turn = base_turn * rng.uniform(0.9, 1.1, len(df))

df["revenue_monthly_tnd"] = (
    df["assets_value_tnd"].values
    * turn
    * (0.90 + 0.20 * density_norm.values)     # density effect ±10%
    * (0.90 + 0.20 * df["maturity"].values)   # maturity effect ±10%
).clip(250, 150000)

# ----------------------------
# 5) Revenue bucket (quantiles => robust)
# ----------------------------
df["revenue_bucket"] = pd.qcut(df["revenue_monthly_tnd"], q=3, labels=["low", "medium", "high"]).astype(str)

# ----------------------------
# 6) Open at night (activity + slight density)
# ----------------------------
base_night = {"kiosk": 0.35, "cafe": 0.70, "grocery": 0.40, "clothing": 0.15, "pharmacy": 0.10}
p_night = df["activity_type"].map(base_night).astype(float).values
p_night = np.clip(p_night + 0.05*(density_norm.values - 0.5), 0.05, 0.85)
df["open_at_night"] = rng.binomial(1, p_night, len(df)).astype(bool)

# ----------------------------
# 7) Security (assets + maturity, very simple)
# ----------------------------
assets = df["assets_value_tnd"].values
m = df["maturity"].values

# alarm probability increases with assets and maturity
p_alarm = 1 / (1 + np.exp(-(-3.0 + 0.00006*assets + 1.2*m)))
p_cam   = 1 / (1 + np.exp(-(-3.8 + 0.00006*assets + 1.0*m)))

df["security_alarm"]  = rng.binomial(1, np.clip(p_alarm, 0.02, 0.95)).astype(bool)
df["security_camera"] = rng.binomial(1, np.clip(p_cam,   0.01, 0.90)).astype(bool)

# fire extinguisher: mostly activity + maturity
base_fire = {"kiosk": 0.20, "cafe": 0.35, "grocery": 0.30, "clothing": 0.25, "pharmacy": 0.55}
p_fire = df["activity_type"].map(base_fire).astype(float).values
p_fire = np.clip(p_fire + 0.20*m, 0.05, 0.90)
df["fire_extinguisher"] = rng.binomial(1, p_fire, len(df)).astype(bool)

# ----------------------------
# 😎 Labels (keep unchanged)
# ----------------------------
df["claim_occurred"] = (df["ClaimNb"] > 0).astype(int)
df["claim_cost_tnd"] = df["ClaimAmount"].astype(float)

# ----------------------------
# 9) Keep only the useful columns (optional)
# ----------------------------
keep_cols = [
    "policy_id" if "policy_id" in df.columns else "IDpol",
    "governorate" if "governorate" in df.columns else "Region",
    "years_active",
    "activity_type",
    "shop_area_m2",
    "assets_value_tnd",
    "revenue_monthly_tnd",
    "revenue_bucket",
    "open_at_night",
    "security_alarm",
    "security_camera",
    "fire_extinguisher",
    "claim_occurred",
    "claim_cost_tnd"
]

keep_cols = [c for c in keep_cols if c in df.columns]
business_df = df[keep_cols].copy()

business_df.head()

,IDpol,Region,years_active,activity_type,shop_area_m2,assets_value_tnd,revenue_monthly_tnd,revenue_bucket,open_at_night,security_alarm,security_camera,fire_extinguisher,claim_occurred,claim_cost_tnd
0,24952.0,R24,0.05,grocery,60.293851,41636.955278,4749.817913,medium,False,False,True,True,0,0.0
1,24953.0,R24,0.67,grocery,55.645702,51021.477592,6653.990820,high,False,False,False,False,0,0.0
2,24955.0,R53,0.89,grocery,75.743936,59667.863956,8224.172273,high,False,True,True,True,0,0.0
3,24956.0,R53,0.10,grocery,67.018591,51336.243377,6071.250481,medium,False,True,False,True,0,0.0
4,24958.0,R24,0.20,grocery,51.717159,34968.809587,3748.563655,low,False,False,False,True,0,0.0


In [68]:
business_df.to_csv("../data/processed/policy_master_business.csv", index=False)


# Adagio 

In [12]:
import pandas as pd

# 1. Charger les données INS
df_ins = pd.read_excel('../data/raw/ins/Nombre Ménage Gouv.xlsx', engine='openpyxl')

# 2. Garder uniquement les colonnes utiles et les renommer proprement
df_ins = df_ins[['Gouvernorat', 'Valeur']].copy()
df_ins.columns = ['governorate', 'nb_menages']

# 3. NETTOYAGE CRUCIAL (pour que 'Tunis' matche avec 'Tunis ')
df_ins['governorate'] = df_ins['governorate'].astype(str).str.strip()

# 4. Ajouter la superficie pour calculer la densité
superficies = {
    'Tunis': 288, 'Ariana': 482, 'Ben Arous': 761, 'Manouba': 1137,
    'Nabeul': 2788, 'Zaghouan': 2768, 'Bizerte': 3685, 'Béja': 3558, # Note l'accent sur Béja
    'Jendouba': 3102, 'Le Kef': 4965, 'Siliana': 4642, 'Kairouan': 6712,
    'Kasserine': 8065, 'Sidi Bouzid': 6994, 'Sousse': 2621, 'Monastir': 1019,
    'Mahdia': 2966, 'Sfax': 7545, 'Gafsa': 8990, 'Tozeur': 4719,
    'Kebili': 22084, 'Gabès': 7175, 'Médenine': 8588, 'Tataouine': 38889
}
df_surf = pd.DataFrame(list(superficies.items()), columns=['governorate', 'area_km2'])

# 5. Fusionner et calculer la densité
df_ins = df_ins.merge(df_surf, on='governorate', how='inner')
df_ins['household_density'] = df_ins['nb_menages'] / df_ins['area_km2']

# 6. Maintenant, fusionne avec ton dataset principal (Small Business)
# df_final = df_votre_data.merge(df_ins[['governorate', 'household_density']], on='governorate', how='left')

print("Succès ! Voici un aperçu de ta densité par gouvernorat :")
print(df_ins[['governorate', 'household_density']].sort_values(by='household_density', ascending=False).head())

Succès ! Voici un aperçu de ta densité par gouvernorat :
   governorate  household_density
0        Tunis        1251.152778
1       Ariana         430.039419
2    Ben Arous         288.176084
11    Monastir         166.420020
3      Manouba         106.501319


In [13]:
df_ins.to_csv("../data/processed/policy_master_ins.csv", index=False)

In [ ]:
# Données de superficie (km2) pour le Grand Tunis, Sousse et Sfax
data_geo = {
    "Tunis": {"area": 288, "poi_count": 30562},      # Ta valeur reçue
    "Ariana": {"area": 482, "poi_count": 2278},         # À remplir avec Overpass
    "Ben Arous": {"area": 761, "poi_count": 2362},      # À remplir avec Overpass
    "Manouba": {"area": 1137, "poi_count": 796},       # À remplir avec Overpass
    "Sfax": {"area": 7545, "poi_count": 2557},          # À remplir avec Overpass
    "Sousse": {"area": 2621, "poi_count": 186}         # À remplir avec Overpass
}

# Calcul automatique de la densité
for gov, values in data_geo.items():
    if values["poi_count"] > 0:
        values["poi_density"] = values["poi_count"] / values["area"]
    else:
        values["poi_density"] = 0

print(data_geo)

{'Tunis': {'area': 288, 'poi_count': 5694, 'poi_density': 19.770833333333332}, 'Ariana': {'area': 482, 'poi_count': 581, 'poi_density': 1.2053941908713692}, 'Ben Arous': {'area': 761, 'poi_count': 490, 'poi_density': 0.6438896189224704}, 'Manouba': {'area': 1137, 'poi_count': 77, 'poi_density': 0.06772207563764292}, 'Sfax': {'area': 7545, 'poi_count': 647, 'poi_density': 0.08575215374420146}, 'Sousse': {'area': 2621, 'poi_count': 186, 'poi_density': 0.07096528042731781}}


In [15]:
import numpy as np

# On applique le log pour lisser l'écart entre Tunis et le reste
for gov in data_geo:
    data_geo[gov]['poi_density_log'] = np.log1p(data_geo[gov]['poi_density'])

# Tunis passera de 19.7 à ~3.0
# Manouba passera de 0.06 à ~0.05
# Les écarts deviennent "traitables" par une IA.